# **Probabilistic Programming Project**

Ghadamiyan Lida

---

The purpose of this project is to implement the LDA algorithm using PyMC, following the indications from the course.





In [ ]:
import pandas as pd


corpus = ["basketball team sport team player compete objective shoot basketball", 
          "player basketball team point game win", 
          "center general tall basketball player tall basketball player forward",
          "friends proffesional basketball player like dennis rodman",
          
          "art diverse range human activiti involv creat visual audit perform artifact",
          "activities relat production work art include art criticism history art",
          "three classical branche visual art paint sculpture architecture",
          "music theatre film dance performing art literature includ broader definition art"]


##**Data Preprocessing**



* Tokenization. 
* Stopwords removal - done by CountVectorizer in feature extraction. 
* Lowercase the words. 
* Remove punctuation.
* Remove short words.
* Lematization - reducing words to their meaningful base form.
* Stemming — reducing words to their base form.
* **Feature Extraction**



In [ ]:
import string
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
nltk.download('wordnet')
nltk.download('punkt')

stemmer = SnowballStemmer('english')

def preprocessing(corpus):

    df = pd.DataFrame({'doc':corpus})     # Converting to data frame

    data = []
    for i in range(0, len(df.index)):

        table = str.maketrans(dict.fromkeys(string.punctuation))                    # Punctuation removal
        words = (df.doc[i].translate(table)).lower() 

        words = nltk.word_tokenize(words) # Tokenization

        words_ = []
        for word in words:
            if len(word) > 2:                                                       # Short words removal
                word1 = stemmer.stem(WordNetLemmatizer().lemmatize(word, pos='v'))  # Lemmatization and stemming           
                words_.append(word1)
        data.append(words_)

    corpus = pd.DataFrame({'doc':data})                                             # Saving as dataframe
    return corpus, data

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data, data_ = preprocessing(corpus)

# **Feature Extraction**

* Bag of Words - CountVectorizer is used for building a dictionary of features, and also for tokenizing and filtering stopwords. 
* Frequencies - We compute the term frequencies by  dividing the number of occurrences given by CountVectorizer by the total number of words.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def feat_extraction(data):
    data = data['doc'].astype(str).values.tolist() # Converting to list of list
    count_vect =  CountVectorizer()
    occurrence = count_vect.fit_transform(data)

    return count_vect, occurrence

In [ ]:
voc, occ = feat_extraction(data)

#print(voc.get_feature_names())
#print(voc.vocabulary_)

vocab_ = {v : k for k, v in voc.vocabulary_.items()}
print(vocab_)


{5: 'basketbal', 43: 'team', 41: 'sport', 32: 'player', 10: 'compet', 29: 'object', 40: 'shoot', 33: 'point', 20: 'game', 47: 'win', 8: 'center', 21: 'general', 42: 'tall', 18: 'forward', 19: 'friend', 35: 'proffesion', 26: 'like', 15: 'denni', 38: 'rodman', 2: 'art', 16: 'divers', 36: 'rang', 23: 'human', 0: 'activ', 25: 'involv', 11: 'creat', 46: 'visual', 4: 'audit', 31: 'perform', 3: 'artifact', 37: 'relat', 34: 'product', 48: 'work', 24: 'includ', 12: 'critic', 22: 'histori', 45: 'three', 9: 'classic', 6: 'branch', 30: 'paint', 39: 'sculptur', 1: 'architectur', 28: 'music', 44: 'theatr', 17: 'film', 13: 'danc', 27: 'literatur', 7: 'broader', 14: 'definit'}


In [ ]:
def frec(occ): 
    data_occ= []
    for doc in occ.toarray():
        words = []
        for poz, n in enumerate(doc):
            for j in range(n):
                words.append(poz)
        data_occ.append(words)


    data_tf = []
    for doc in data_occ:
        doc_tf = []
        for w in doc:
            doc_tf.append(w / len(vocab_))
        data_tf.append(doc_tf)
    return data_tf, data_occ


After I tried both with data and data_tf, I concluded that working with frequencies indead of occurences gives better resoults.

In [ ]:
data, data_occ = frec(occ)
print(data_occ)

[[5, 5, 10, 29, 32, 40, 41, 43, 43], [5, 20, 32, 33, 43, 47], [5, 5, 8, 18, 21, 32, 32, 42, 42], [5, 15, 19, 26, 32, 35, 38], [0, 2, 3, 4, 11, 16, 23, 25, 31, 36, 46], [0, 2, 2, 2, 12, 22, 24, 34, 37, 48], [1, 2, 6, 9, 30, 39, 45, 46], [2, 2, 7, 13, 14, 17, 24, 27, 28, 31, 44]]


# **Latent Dirichlet Allocation**

LDA is a statistical model that reflects the belonging of a document to several topics.

We are usig CompleteDirichlet instead of Dirichlet because of its property to assign to the last element the rest of the sum to one.

Alpha and beta reprezents the priors and are initialized with one.

Let t be the number of topics and  d the size of the corpus. Thus, the LDA generative process is:

1. For each topic: 

 a) Draw a distribution over words $\phi_d = Dirichlet(\beta)$


2. For each document: 

 a) Draw a topic of vector proportions $\theta_t = Dirichlet(\alpha)$
        
 b) For each word: 

    i) Draw a topic assignment $z_{d, t} = Multinomial(\theta_d)$
        
    ii) Draw a word $w_{d, t} = Multinomial(\phi_{z_{t, d}})$
  




In [ ]:
!pip install pymc

In [ ]:
import pymc as pm
import numpy as np

nr_topics = 2  
vocab_size = len(vocab_)
corpus_size = len(data)

alpha = np.ones(nr_topics)*0.5
beta = np.ones(vocab_size)*0.5
Nm = [len(doc) for doc in data]

phi_ = pm.Container([pm.Dirichlet("phi_ %s" % topic, theta = beta) for topic in range(nr_topics)])

phi = pm.Container( [pm.CompletedDirichlet("Phi %s" % topic,  phi_[topic])  for topic in range(nr_topics)] ) #word distribution per topic

theta_ = pm.Container([pm.Dirichlet("theta_ %s" % doc, theta = alpha) for doc in range(corpus_size)])

theta = pm.Container([pm.CompletedDirichlet("Theta %s" % doc, theta_[doc]) for doc in range(corpus_size)])    # topic distribution per docs


z = pm.Container([pm.Categorical('Z %i' % doc,       # topic for word per docs
                             p = theta[doc], 
                             size = Nm[doc],
                             value = np.random.randint(nr_topics, size = Nm[doc]))
                for doc in range(corpus_size)])


w = pm.Container([pm.Categorical("W %i %i" % (doc, word),     # the word from doc
                                p = pm.Lambda('Phi Z %i %i' % (doc, word), 
                                             lambda z = z[doc][word], 
                                             phi = phi: phi[z]),
                                value = data[doc][word], 
                                observed = True)
                for doc in range(corpus_size)
                for word in range(Nm[doc]) ])


model = pm.Model([phi_, theta_, theta, phi, z, w])

map_ = pm.MAP(model) # improving convergence
map_.fit()

mcmc = pm.MCMC(model)    # fitting
tr = mcmc.sample(1000, 400)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


/usr/local/lib/python3.6/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


 [-----------------100%-----------------] 1000 of 1000 complete in 1.3 sec

In [ ]:
print('Topic distribution for each word:\n')
for doc in range(corpus_size):  # topic distribution per word per document
    print(mcmc.trace('Z %i' % doc)[-1]) 

Topic distribution for each word:

[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
print('Topic distribution per document:\n')
for doc in range(corpus_size):  # topic distribution per document
    print(mcmc.trace('Theta %i' % doc)[0]) 

Topic distribution per document:

[[1.000000e+00 1.538214e-12]]
[[1.00000000e+00 7.58637597e-12]]
[[1.00000000e+00 7.94231347e-12]]
[[1.00000000e+00 1.14478427e-11]]
[[1.00000000e+00 6.23534557e-12]]
[[2.92530819e-11 1.00000000e+00]]
[[1.01291382e-11 1.00000000e+00]]
[[1.00000000e+00 1.27328148e-11]]


In [ ]:
def topics__(nr_topics, vocab_, w):
    
    for topic in range(nr_topics):
        print ("Topic % i" % topic)
        
        idxs = np.argsort(mcmc.trace('Phi %i' % topic)[:].mean(axis=0)[0], axis = 0)   # trace - iid draws from the posterior 
        words_ = [vocab_[idx] for idx in idxs]

        list_ = []

        for i, j in enumerate(mcmc.trace('Phi %i' % topic)[:].mean(axis=0)[0]):
            for id in idxs:
                if id == i:
                    list_.append(j)

        for ix in idxs[w:0:-1]:
            print("\t", words_[ix], ":", list_[ix])


In [ ]:
topics__(nr_topics, vocab_,50)

Topic  0
	 forward : 0.1114883735174976
	 work : 0.0771856823585537
	 divers : 0.07340999710800894
	 compet : 0.07324545134127951
	 branch : 0.07218832590328363
	 rang : 0.07079286527958571
	 visual : 0.0565536940896106
	 classic : 0.05621288795361776
	 histori : 0.053191279235019116
	 like : 0.03949055419212663
	 perform : 0.035381638423640366
	 point : 0.03442343620222972
	 theatr : 0.025470167424602508
	 involv : 0.018386331474519128
	 proffesion : 0.016295045891610006
	 team : 0.016242178345574114
	 creat : 0.01597529672229588
	 paint : 0.015058085437278417
	 architectur : 0.015016878264685122
	 literatur : 0.01407624961973752
	 shoot : 0.010549267409372731
	 basketbal : 0.01039260322570474
	 tall : 0.010174268239288629
	 three : 0.009059318030847906
	 game : 0.008860286229730075
	 danc : 0.007932262103898852
	 activ : 0.007437828541845352
	 player : 0.005973253468021714
	 human : 0.005684990361801037
	 art : 0.005610168500504996
	 film : 0.005040808537219693
	 music : 0.0042641900

# **TASK2**

# Can the topic model be used to define a topic-based similarity measure between documents?

We will analyze the similariy between topics by computing the cosine distance and JensenShannon distance of the theta distribution (i.e. the distribution of topics per document) of the first n documents given

In [ ]:
from scipy.spatial import distance

def cos_sim(nr_doc):

    print('Documents \t cosine similarity \t JensenShannon similarity')
    for i in range(nr_doc):
        for j in range(nr_doc):
            if i != j:
                print('%i & %i:\t\t %f \t\t %f ' %(i,j,
                                             1-distance.cosine( mcmc.trace('Theta %i' % i)[:].mean(axis=0)[0] , mcmc.trace('Theta %i' % j)[:].mean(axis=0)[0]),
                                             1-distance.jensenshannon( mcmc.trace('Theta %i' % i)[:].mean(axis=0)[0] , mcmc.trace('Theta %i' % j)[:].mean(axis=0)[0])))       
                

In [ ]:
print('Topic similarity between documents:\n')
cos_sim(6)

Topic similarity between documents:

Documents 	 cosine similarity 	 JensenShannon similarity
0 & 1:		 1.000000 		 0.999999 
0 & 2:		 1.000000 		 0.999999 
0 & 3:		 1.000000 		 0.999999 
0 & 4:		 1.000000 		 0.999999 
0 & 5:		 0.000000 		 0.167445 
1 & 0:		 1.000000 		 0.999999 
1 & 2:		 1.000000 		 1.000000 
1 & 3:		 1.000000 		 1.000000 
1 & 4:		 1.000000 		 1.000000 
1 & 5:		 0.000000 		 0.167445 
2 & 0:		 1.000000 		 0.999999 
2 & 1:		 1.000000 		 1.000000 
2 & 3:		 1.000000 		 1.000000 
2 & 4:		 1.000000 		 1.000000 
2 & 5:		 0.000000 		 0.167445 
3 & 0:		 1.000000 		 0.999999 
3 & 1:		 1.000000 		 1.000000 
3 & 2:		 1.000000 		 1.000000 
3 & 4:		 1.000000 		 0.999999 
3 & 5:		 0.000000 		 0.167445 
4 & 0:		 1.000000 		 0.999999 
4 & 1:		 1.000000 		 1.000000 
4 & 2:		 1.000000 		 1.000000 
4 & 3:		 1.000000 		 0.999999 
4 & 5:		 0.000000 		 0.167445 
5 & 0:		 0.000000 		 0.167445 
5 & 1:		 0.000000 		 0.167445 
5 & 2:		 0.000000 		 0.167445 
5 & 3:		 0.000000 		 0.167445 
5 & 4:	

# **What about a new document? How can topics be assigned to it?**

If we are given a new document, we can compare the probability of the document belonging to topic 1 with the probability of the document belonging to topic 2 using Bayes Theorem.

Thus, we want to compute $F = \frac{P(\theta_1|NewD)}{P(\theta_2|NewD)}$, knowing that $ P(\phi_i|NewD) = \frac{P(NewD|\theta_i)P(\theta_i)}{P(NewD)} = \frac{ \prod_{j} P(NewD_j|\theta_i)P(\theta_i)}{P(NewD)} $. 

Therefor, $F = \frac{ \prod_{j} P(NewD_j|\theta_1)P(\theta_1)}{\prod_{j} P(NewD_j|\theta_2)P(\theta_2)}$.

$P(\theta_i) = $

In [ ]:
# Computing P(Ti)
pti = []
for topic in range(nr_topics):
    pp = 1
    for j in mcmc.trace('Phi %i' % topic)[:].mean(axis=0)[0]:
        pp = pp*j
    
    pti.append(pp)
        


In [ ]:
print(pti)

[3.157017892853357e-117, 8.602118166862097e-131]


In [ ]:
NewD = ['Basketball player tall art dance']

dataN, data_N = preprocessing(NewD)
vocN, occN = feat_extraction(dataN)

In [ ]:
dataN, data_occN = frec(occN)
print(dataN)

[[0.0, 0.02040816326530612, 0.04081632653061224, 0.061224489795918366, 0.08163265306122448]]


In [ ]:
vocab_N = {v : k for k, v in vocN.vocabulary_.items()}

In [ ]:
print(vocab_N)

{1: 'basketbal', 3: 'player', 4: 'tall', 0: 'art', 2: 'danc'}


In [1]:
basketbal0 = 0.01039260322570474
tall0 = 0.010174268239288629
danc0 = 0.007932262103898852
player0 = 0.005973253468021714
art0 = 0.005610168500504996

art1 = 0.11591236971001206
basketbal1 = 0.005280753560451351
player1 = 0.003618249327591763
danc1 = 0.003219895269802611
tall1 = 0.021419923561106554

In [2]:
p_newd_t0 = basketbal0*art0*danc0*player0*tall0*pti[0]
p_newd_t1 = basketbal1*art1*danc1*player1*tall1*pti[1]

In [3]:
if p_newd_t0 > p_newd_t1:
    print('NewD has topic 0 ')
else:
    print('NewD has topic 1 ')

NewD has topic 0 
